In [ ]:
# -------------------------------------------------------------------- aula 2 --------------------------------------------------------------------

In [ ]:
# importando biblioteca
import pandas as pd
import plotly.express as px

In [ ]:
# ----------------------------------------------------- importando todas as planilhas -----------------------------------------------------

In [ ]:
# Principal
planilha_principal = pd.read_excel("tabela_de_acoes.xlsx", sheet_name="Principal")
display(planilha_principal)

In [ ]:
# Chatgpt  
planilha_chatgpt = pd.read_excel("tabela_de_acoes.xlsx", sheet_name="chatgpt")
display(planilha_chatgpt)

In [ ]:
# Total de Ações
planilha_total_de_acoes = pd.read_excel("tabela_de_acoes.xlsx", sheet_name="Total_de_acoes")
display(planilha_total_de_acoes)

In [ ]:
# Ticker
planilha_ticker = pd.read_excel("tabela_de_acoes.xlsx", sheet_name="Ticker")
display(planilha_ticker)

In [ ]:
# -------------------------------------------------------------------- aula 3 --------------------------------------------------------------------

In [ ]:
# Selecionando apenas as colunas necessarias da planilha principal
planilha_principal = planilha_principal[['Ativo', 'Data', 'Último (R$)', "Var. Dia (%)"]].copy()
display(planilha_principal)

In [ ]:
# Renomeando as colunas com caracteres especiais
planilha_principal = planilha_principal.rename(columns={'Último (R$)':'valor_final', "Var. Dia (%)":"var_dia"}).copy()
display(planilha_principal)

In [ ]:
# Porcentuando a coluna var_dia e criando nova coluna valor_inicial
planilha_principal['var_dia'] = planilha_principal['var_dia']/100
planilha_principal["valor_inicial"] = planilha_principal['valor_final'] / (planilha_principal['var_dia'] + 1)
display(planilha_principal)

In [ ]:
# Puxando coluna "Qtde. Teorica" da planilha Total de acoes para a principal 
planilha_principal = planilha_principal.merge(planilha_total_de_acoes, left_on='Ativo', right_on='Código', how='left').drop(columns=["Código"])
display(planilha_principal)

In [ ]:
# Criando coluna variacao
planilha_principal['variacao'] = (planilha_principal['valor_final'] - planilha_principal['valor_inicial']) * planilha_principal['Qtde. Teorica']
display(planilha_principal)

In [ ]:
# Criando coluna Resultado
planilha_principal['Resultado'] = planilha_principal['variacao'].apply(lambda x: "Subiu" if x > 0 else ("Desceu" if x < 0 else "Estável"))

display(planilha_principal)

In [ ]:
# Puxando coluna "Nome" da planilha Ticker para a Principal
planilha_principal = planilha_principal.merge(planilha_ticker, left_on='Ativo', right_on='Ticker', how='left').drop(columns=["Ticker"])

display(planilha_principal)

In [ ]:
# Puxando coluna Segmento e Idade da planilha Chatgpt para planilha Principal
planilha_principal = planilha_principal.merge(planilha_chatgpt, left_on='Nome', right_on='Ativo', how='left').drop(columns=["Ativo_y"])
display(planilha_principal)


In [ ]:
# ------------------------------------------------------------------- Análise -------------------------------------------------------------------

# função para formatar dinheiro ----------------------------------------------
def format(valor, name):
    valor_formatado = 'R$ {:,.2f}'.format(valor)
    display(f'{name}: {valor_formatado}')


analise = {}
# Maior
analise['Maior'] = planilha_principal['variacao'].max()
format(analise['Maior'], 'Maior')

# Menor
analise['Menor'] = planilha_principal['variacao'].min()
format(analise['Menor'], 'Menor')

# Média
analise['Media'] = planilha_principal['variacao'].mean()
format(analise['Media'], 'Media')

# Média de quem subiu
analise['Media de quem subiu'] = planilha_principal[planilha_principal['Resultado'] == "Subiu" ]['variacao'].mean()
format(analise['Media de quem subiu'], 'Media de quem subiu')

# Média de quem desceu
analise['Media de quem desceu'] = planilha_principal[planilha_principal['Resultado'] == "Desceu" ]['variacao'].mean()
format(analise['Media de quem desceu'], 'Media de quem desceu')

In [ ]:
subiu = planilha_principal[planilha_principal['Resultado'] == "Subiu" ]
display(subiu)

In [73]:
analise_segmento = subiu.groupby("Segmento")["variacao"].sum().reset_index()
display(analise_segmento) 

,Segmento,variacao
0,Alimentos Processados,4.078337e+08
1,Comércio,5.412724e+08
2,Construção Civil,3.752587e+07
3,Educação,7.229584e+07
4,Energia Elétrica,6.823156e+08
5,Energia e Logística,1.388008e+08
6,Exploração de Imóveis,1.806845e+07
7,Holding,3.738540e+08
8,Intermediários Financeiros,4.636680e+09
9,Investimentos,4.223825e+07


In [77]:
analise_saldo = planilha_principal.groupby("Resultado")["variacao"].sum().reset_index()
display(analise_saldo)

,Resultado,variacao
0,Desceu,-6.338820e+09
1,Estável,0.000000e+00
2,Subiu,2.585859e+10


In [85]:
fig_saldo = px.bar(analise_saldo, x='Resultado', y='variacao', text='variacao', title="Variação", color="variacao",text_auto='.3s')
fig_saldo.show()

In [86]:
fig_segmento = px.pie(analise_segmento, names='Segmento', values='variacao', title="Variação")
fig_segmento.show()